In [12]:
from model import sp_syllabler
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.metrics.distance import edit_distance

In [40]:
# 1
# evaluation for ox
with open('data/ox/x_val_ortho.pkl', 'rb') as file:
    x_val_ortho = pickle.load(file)

with open('data/ox/y_val.pkl', 'rb') as file:
    y_val = pickle.load(file)

In [ ]:
with open('data/x_val_ortho.pkl', 'rb') as file:
    x_val_ortho = pickle.load(file)
    
with open('data/x_val_ipa.pkl', 'rb') as file:
    x_val_ipa = pickle.load(file)

with open('data/y_val.pkl', 'rb') as file:
    y_val = pickle.load(file)
    
with open('data/x_tr_ortho.pkl', 'rb') as file:
    x_tr_ortho = pickle.load(file)

In [41]:
# 2
# set up for mw eval
# ortho max encoder len = 45
# ipa max encoder len = 74

with open('data/ox/e2i_ortho.pkl', 'rb') as file:
    e2i_ortho = pickle.load(file)

shitter = sp_syllabler(e2i_ortho= e2i_ortho, ortho_input_size=20,latent_dim=32,embed_dim=32 ,max_feat=36)
shitter.model.load_weights('data/ox/single_pen_best_weights.h5')

In [ ]:
#ortho max encoder len = 45
#ipa max encoder len = 74

with open('data/e2i_ortho.pkl', 'rb') as file:
    e2i_ortho = pickle.load(file)
    
with open('data/e2i_ipa.pkl', 'rb') as file:
    e2i_ipa = pickle.load(file)

double_penetrator = dp_syllabler(e2i_ortho= e2i_ortho, e2i_ipa = e2i_ipa, ortho_input_size=45,ipa_input_size=74,latent_dim=32,embed_dim=32 ,max_feat=148)
double_penetrator.model.load_weights('data/32_false_double_pen_best_weights.h5')

In [66]:
def calc_brier(attempted, probability):
    total = 0
    sum_brier = 0
    for word in attempted:
        for c in attempted:
            total += 1
            if c == 2:
                sum_brier += (probability - 1)**2
            elif c == 1:
                sum_brier += (probability - 0)**2
    return (1./total)*(sum_brier)

In [64]:
# 3

def calc_f1(attempted, true, filename = 'data/ox/evaluation_output_single_pen.txt'):
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    correct_num_char = 0
    total_checked = 0
    evaluation_file = open(filename, 'w+')
    for i in range(0, len(attempted)):
        total_checked += 1
        if (len(attempted[i]) == len(true[i])):
            correct_num_char += 1
            for j in range(0, len(attempted[i])):
                if(attempted[i][j] == true[i][j]):
                    if true[i][j] == 1:
                        true_neg += 1
                    elif true[i][j] == 2:
                        true_pos += 1
                else:
                    if true[i][j] == 1:
                        false_pos += 1
                    elif true[i][j] == 2:
                        false_neg += 1
    evaluation_file.write("Total checked: " + str(total_checked) + '\n')
    evaluation_file.write("Num correctly evaluated character count: " +  str(correct_num_char) + '\n')
    evaluation_file.write("True positives: " + str(true_pos) + '\n')
    evaluation_file.write("True negatives: " + str(true_neg) + '\n')
    evaluation_file.write("False positives: " + str(false_pos) + '\n')
    evaluation_file.write("False negatives: " + str(false_neg) + '\n')
    brier = calc_brier(attempted, probability=0.2)
    precision = true_pos/(true_pos+false_pos)
    recall = true_pos/(true_pos + false_neg)
    f_one = 2/((1/precision)+(1/recall))
    evaluation_file.write("precision: " + str(precision) + '\n')
    evaluation_file.write("recall: " + str(recall) +  '\n')
    evaluation_file.write("F1 scores: " + str(f_one) + '\n')
    evaluation_file.write("Brier score: " + str(brier) + '\n')
    evaluation_file.close()

In [43]:
# 4

reals = []
for x in y_val:
    reals += [[i for i in x if i !=0]]

In [44]:
print(len(x_val_ortho))

6500


In [45]:
# 5
# running through all data points

attempts = []
for i in range(0, len(x_val_ortho)):
    attempts += [shitter.raw_syllabify(x_val_ortho[i])]
    print(i, end='\r')

In [46]:
# 6

attempts_stripped = []
for x in attempts:
    attempts_stripped += [[i for i in x if i !=0]]

In [47]:
# 7

def inconsistency_grab(attempts, reals, filename = 'data/ox/incorrect_syls.txt'):
    global converted_back_to_eng
    sum_lev = 0
    incorrect_counter = 0
    incorrect_syl_count = 0
    file = open(filename, 'w+', encoding='utf-8')
    file.write('Attempt' + '\t' + 'Real' + '\n')
    for i in range(0, len(attempts)):
        if attempts[i] != reals[i]:
            incorrect_counter += 1
            a = shitter.insert_syl(converted_back_to_eng[i], attempts[i])
            r = shitter.insert_syl(converted_back_to_eng[i], reals[i])
            if len(a.split('-')) != len(r.split('-')):
                incorrect_syl_count += 1
            sum_lev += edit_distance(a,r,substitution_cost=1, transpositions=True)
            file.write(a + '\t' + r + '\n')
    file.write("Words with errors: %i"%incorrect_counter +'\n')
    file.write("Words with incorrect number of syllables: %i"%incorrect_syl_count +'\n')
    file.write("Total evluated: %i"%len(attempts) +'\n')
    file.write("Perfect accuracy: %.2f"%((len(attempts) - incorrect_counter)/len(attempts)) +'\n')
    file.write("Number of syllables accuracy: %.2f"%((len(attempts) - incorrect_syl_count)/len(attempts)) +'\n')
    file.write("Average Levenshtein Distance(across incorrect words): %.2f"%(sum_lev/incorrect_counter) +'\n')
    file.close()

In [48]:
# 8

converted_back_to_eng = []
for x in x_val_ortho:
    real_word = ""
    for i in x:
            if i != 0:
                real_word += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    converted_back_to_eng += [real_word]

In [49]:
# 9 converting attempts to english with hyphens

eng_conv_attempts = []
for i in range(0, len(attempts)):
    eng_conv_attempts += [shitter.insert_syl(converted_back_to_eng[i], attempts[i])]

In [67]:
# 10 calculation call mw

calc_f1(attempts_stripped, reals)
inconsistency_grab(attempts_stripped, reals, 'data/ox/incorrect_syls.txt')

In [ ]:
calc_f1(attempts_stripped, reals, 'data/32_pure_false_evaluation_output.txt')
inconsistency_grab(attempts_stripped, reals, 'data/32_incorrect_syls.txt')

In [ ]:
# testing with falses
attempts_prime = []
for i in range(0, 5000):
    attempts_prime += [double_penetrator.raw_syllabify(x_val_ortho[i], np.array(false_ipa[i]))]
    print(i, end='\r')
    
attempts_prime_stripped = []
for x in attempts_prime:
    attempts_prime_stripped += [[i for i in x if i !=0]]
    
calc_f1(attempts_prime_stripped, reals, 'data/falses_evaluation_output_double_pen.txt')

In [ ]:
with open('data/eng_words.txt','w+') as file:
    for i in range(0,5000):
        file.write(converted_back_to_eng[i] + '\n')

In [ ]:
# falsing training data
tr_converted_back_to_eng = []
for x in x_tr_ortho:
    real_word = ""
    for i in x:
            if i != 0:
                real_word += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    tr_converted_back_to_eng += [real_word]


false_ipa_orig = []
false_ipa = []

for i in range(0,len(x_tr_ortho)):
    tried = []
    for q in tr_converted_back_to_eng[i]:
        tried += l2i[q]
    false_ipa_orig += [tried]
    
for x in false_ipa_orig:
    inted = []
    for y in x:
        inted += [e2i_ipa[y]]
    false_ipa += [inted]
        
    
false_ipa = pad_sequences(false_ipa, maxlen=74, padding='post')

with open('data/false_ipa.pkl', 'wb+') as file:
    pickle.dump(false_ipa, file)

In [ ]:
# 0

l2i = {'a':'æ', 'b':'b', 'c':'k', 'd':'d', 'e':'e', 'f':'f' , 'g':'g', 'h':'h', 'i':'ɪ', 'j':'ʒ', 'k':'k', 'l':'l', 'm':'m', 'n':'n',
      'o':'ɑ', 'p':'p', 'q':'k', 'r':'ɹ', 's':'s', 't':'t', 'u':'ə', 'v':'v', 'w':'w', 'x':['k','s'], 'y':'j', 'z':'z'}

In [61]:
length_mismatches = []
mismatch_indexes = []

for i in range(0, len(attempts_stripped)):
    if len(attempts_stripped[i]) != len(reals[i]):
        print(len(attempts_stripped[i]))
        print(len(reals[i]))
        print('---------------')
        mismatch_indexes += [i]
print(len(mismatch_indexes))

7
6
---------------
12
6
---------------
9
7
---------------
11
6
---------------
11
12
---------------
6
7
---------------
8
10
---------------
5
4
---------------
8
7
---------------
15
6
---------------
13
11
---------------
11


In [62]:
for x in mismatch_indexes[:10]:
    print(converted_back_to_eng[x])
    print(len(attempts_stripped[x]))
    print(len(reals[x]))
    print(attempts_stripped[x])
    print(eng_conv_attempts[x])
    print('------------')

signers
7
6
[1, 1, 2, 1, 1, 1, 1]
sig-ners
------------
typographers
12
6
[1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1]
ty-pog-ra-ph-ers
------------
attempted
9
7
[1, 2, 1, 1, 1, 1, 2, 1, 1]
at-tempt-ed
------------
desirabilia
11
6
[1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1]
de-sir-a-bil-i-a
------------
collectively
11
12
[1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1]
col-lec-tive-ly
------------
collide
6
7
[1, 1, 2, 1, 1, 1]
col-lide
------------
collective
8
10
[1, 1, 2, 1, 1, 2, 1, 1]
col-lec-tive
------------
based
5
4
[1, 1, 1, 1, 1]
based
------------
burnings
8
7
[1, 1, 1, 2, 1, 1, 1, 1]
burn-ings
------------
suburbanisation
15
6
[1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1]
sub-ur-ban-i-sa-tion
------------


12


11
